## 使用GPT_API_free获得密钥

In [10]:
import getpass
import os
import langchain
from langchain.schema.messages import HumanMessage, SystemMessage,AIMessage
from langchain_openai import ChatOpenAI

from openai import OpenAI
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="OPENAI-API-KEY",
    # base_url=https://api.chatanywhere.tech/v1"
)
model = ChatOpenAI(model="gpt-3.5-turbo")

In [11]:
messages = [
    SystemMessage(content="用中文回复"),
    HumanMessage(content="中国有几个省份"),
]
model.invoke(messages)

AIMessage(content='中国共有34个省级行政区，其中包括23个省、5个自治区、4个直辖市和2个特别行政区。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 23, 'total_tokens': 62, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-acfce28e-6dcc-438a-ad56-4b6c3bb6ace3-0', usage_metadata={'input_tokens': 23, 'output_tokens': 39, 'total_tokens': 62, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

# 向量存储和检索器

#### LangChain 实现了一个文档抽象，旨在表示一个文本单元及其相关元数据。它有两个属性：page_content：一个表示内容的字符串；metadata：一个包含任意元数据的字典。

#### metadata 属性可以捕获有关文档来源、与其他文档的关系以及其他信息。请注意，单个 Document 对象通常表示一个较大文档的一部分。

In [12]:
from langchain_core.documents import Document
documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",#包含了关于某个宠物的信息。
        metadata={"source": "mammal-pets-doc"},#包含该文档的元数据，在这里显示的是该文档的来源（source）。
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

#### 在这里生成了五个文档，包含指示三个不同“来源”的元数据。

## 向量存储

#### 向量搜索是一种常见的存储和搜索非结构化数据（例如非结构化文本）的方法。其思想是存储与文本相关联的数值向量。给定一个查询，可以将其嵌入为相同维度的向量，并使用向量相似性度量来识别存储中的相关数据。LangChain 向量存储对象包含用于将文本和文档对象添加到存储中以及使用各种相似性度量进行查询的方法。它们通常使用嵌入模型进行初始化，这决定了文本数据如何转换为数值向量。LangChain 包含与不同向量存储技术的集成套件。一些向量存储由提供商（例如各种云提供商）托管，并需要特定的凭据才能使用；一些（如 Postgres）在可以本地运行或通过第三方运行的独立基础设施中运行；其他可以在内存中运行以处理轻量级工作负载。这里使用Chroma的LangChain 向量存储，它包括一个内存实现。要实例化一个向量存储，通常需要提供一个嵌入模型，以指定文本应如何转换为数值向量。这里我们将使用OpenAI嵌入。

In [13]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
vectorstore = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings(),
)

#### 调用  .from_documents  将把文档添加到向量存储中。向量存储实现了可以在对象实例化后调用的添加文档的方法。大多数实现将允许您连接到现有的向量存储——例如，通过提供客户端、索引名称或其他信息。

#### 一旦实例化了一个包含文档的向量存储，就可以对其进行查询。向量存储包含用于查询的方法：

1.同步和异步；2.通过字符串查询和通过向量；3.有和没有返回相似性分数；4.通过相似性和最大边际相关性（以平衡查询的相似性与检索结果的多样性）。这些方法的输出通常会包含文档对象的列表。

#### 根据与字符串查询的相似性返回文档：在执行相似度搜索时，返回的分数（score）或距离（distance）可能根据不同的提供商（例如 Chroma）而有所不同。具体来说，Chroma 返回的距离度量是与相似度成反比的，也就是说：距离越小，表示查询项与文档之间的相似度越高；距离越大，表示相似度越低。

In [14]:
vectorstore.similarity_search_with_score("cat")

[(Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.37521976232528687),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  0.48278287053108215),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  0.49597978591918945),
 (Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  0.4974355101585388)]

因此可以看出与 page_content='Cats are independent pets that often enjoy their own space.'之间的相似度更高 

#### 异步查询

In [15]:
await vectorstore.asimilarity_search("cat")

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

#### 根据与嵌入查询的相似性返回文档：

In [16]:
embedding = OpenAIEmbeddings().embed_query("cat")
vectorstore.similarity_search_by_vector(embedding)

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

## 检索器

#### LangChain VectorStore对象不继承Runnable，因此无法立即集成到LangChain表达式chains中。LangChain检索器是Runnables，因此它们实现了一组标准方法（例如，同步和异步的invoke和batch操作），并设计为可以纳入LCEL链中。我们可以自己创建一个简单版本，而无需继承Retriever。如果选择希望用于检索文档的方法，可以轻松创建一个可运行的对象。下面围绕 similarity_search 方法构建一个：

In [17]:
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda
retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)  # select top result
retriever.batch(["cat", "shark"])

[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')]]

#### 向量存储实现了一个 as_retriever 方法，该方法将生成一个检索器，特别是一个 VectorStoreRetriever。这些检索器包括特定的 search_type 和search_kwargs 属性，用于识别调用底层向量存储的方法，以及如何对其进行参数化。例如，可以用以下方式复制上述内容：

In [18]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)
retriever.batch(["cat", "shark"])

[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')]]

#### 检索器可以轻松地纳入更复杂的应用程序，例如检索增强生成（RAG）应用程序，这些应用程序将给定问题与检索到的上下文结合成 LLM 的提示。

In [19]:
from langchain_core.prompts import ChatPromptTemplate  #ChatPromptTemplate用来创建与 LLM 交互的提示模板。
from langchain_core.runnables import RunnablePassthrough  #RunnablePassthrough用于简单的无修改传递数据。

llm = ChatOpenAI(model="gpt-3.5-turbo")
message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""
prompt = ChatPromptTemplate.from_messages([("human", message)])
rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

In [20]:
response = rag_chain.invoke("tell me about dog")
print(response.content)

Dogs are great companions, known for their loyalty and friendliness.
